# Lab 2: Working with APIs, JSON, and Web Requests
## Goal: Learn to call REST APIs, pass headers/auth, parse JSON, handle errors/retries, and send GET/POST requests your agents can use.
- Estimated time: 75–100 minutes
- Deliverable: A single script or notebook that completes all steps below.

In [2]:
# Project setup (5 min)

!pip install requests python-dotenv


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
# Step 2:  Hello, GET request (10 min)
# Use a stable public API (JSONPlaceholder) to fetch posts.

import requests, json
 
BASE = "https://jsonplaceholder.typicode.com"
r = requests.get(f"{BASE}/posts/1", timeout=40)
r.raise_for_status()
data = r.json()
print(type(data), data["id"], data["title"])

# Inspect response metadata:

print("Status:", r.status_code)
print("Content-Type:", r.headers.get("content-type"))
print("Preview:", json.dumps(data, indent=2)[:300])

<class 'dict'> 1 sunt aut facere repellat provident occaecati excepturi optio reprehenderit
Status: 200
Content-Type: application/json; charset=utf-8
Preview: {
  "userId": 1,
  "id": 1,
  "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
  "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
}


### 3) Query params + pagination pattern (10–12 min)

In [2]:
# Simulate pagination by limiting results, then iterating pages.

# JSONPlaceholder doesn't paginate, so we emulate pages of size 10

def fetch_posts_page(page:int, size:int=10):
    start = (page-1)*size + 1
    end = start + size - 1
    r = requests.get(f"{BASE}/posts", timeout=10)
    r.raise_for_status()
    all_posts = r.json()
    return all_posts[start-1:end]
 
page1 = fetch_posts_page(1); page2 = fetch_posts_page(2)
print("Page1 len:", len(page1), "| first title:", page1[0]["title"])
print("Page2 len:", len(page2))

#(You’ll replace this with real pagination when using production APIs.)

Page1 len: 10 | first title: sunt aut facere repellat provident occaecati excepturi optio reprehenderit
Page2 len: 10


### 4. Post/Put/Delete

In [7]:
# Practice mutating endpoints (JSONPlaceholder simulates writes).
import requests

BASE = "https://jsonplaceholder.typicode.com"

# Create
new_post = {"title": "Agent Lab", "body": "Hello API", "userId": 42}
created = requests.post(f"{BASE}/posts", json=new_post, timeout=10)
print("Created:", created.status_code)
created_json = created.json()
print(created_json)

# ✅ Limit update to known IDs (1–100)
update_id = created_json.get("id", 1)
if update_id > 100:
    update_id = 1  # fallback for fake API

update = {"title": "Agent Lab v2"}
updated = requests.put(f"{BASE}/posts/{update_id}", json=update, timeout=10)
print("Updated:", updated.status_code)
print("Raw response text:", updated.text)

# Only parse JSON if body exists and code < 400
if updated.ok and updated.text.strip():
    updated_json = updated.json()
    print("Updated JSON:", updated_json)
else:
    print("⚠️ Skipping JSON parse — empty or failed response.")

# Delete
deleted = requests.delete(f"{BASE}/posts/{update_id}", timeout=10)
print("Deleted:", deleted.status_code)
print("Deleted body:", deleted.text)

Created: 201
{'title': 'Agent Lab', 'body': 'Hello API', 'userId': 42, 'id': 101}
Updated: 200
Raw response text: {
  "title": "Agent Lab v2",
  "id": 1
}
Updated JSON: {'title': 'Agent Lab v2', 'id': 1}
Deleted: 200
Deleted body: {}


### 5) Headers & Auth with GitHub API 

In [11]:
import requests

# No token needed for public GitHub API calls
headers = {"Accept": "application/vnd.github+json"}

# Check rate limit info (works without token)
gh = requests.get("https://api.github.com/rate_limit", headers=headers, timeout=10)
print("GitHub status:", gh.status_code)

# Extract and print rate limit info
rate_info = gh.json().get("resources", {}).get("core", {})
print("Rate info:", rate_info)

# Fetch issues from a public repo (e.g., Flask)
owner, repo = "pallets", "flask"
issues = requests.get(
    f"https://api.github.com/repos/{owner}/{repo}/issues",
    headers=headers,
    params={"state": "open", "per_page": 5},
    timeout=10
)
issues.raise_for_status()

# Display top open issues
for issue in issues.json():
    print(f"#{issue['number']} - {issue['title']}")

GitHub status: 200
Rate info: {'limit': 60, 'remaining': 60, 'reset': 1762261921, 'used': 0, 'resource': 'core'}
#5836 - Test failures with click 8.3.1
#5835 - Add Seenode deployment instructions to documentation
#5834 - `pre-commit autoupdate --freeze` 2025-10-23
#5833 - fix typos discovered by  codespell
#5832 - GitHub Actions: Add Python 3.14 and 3.14t to the testing


### 6) Robust error handling + retries (12–15 min)

In [13]:
# Add timeouts, raise_for_status(), and exponential backoff.

import time
 
def robust_get(url, headers=None, params=None, retries=3, backoff=1.5):
    for attempt in range(1, retries+1):
        try:
            r = requests.get(url, headers=headers, params=params, timeout=10)
            if r.status_code == 429:  # rate limited
                wait = int(r.headers.get("Retry-After", 2))
                time.sleep(wait); continue
            r.raise_for_status()
            return r.json()
        except requests.RequestException as e:
            print(f"[Attempt {attempt}] Error: {e}")
            if attempt == retries: raise
            time.sleep(backoff**attempt)
 
data = robust_get(f"{BASE}/posts/2")
print("Robust fetch title:", data["title"])

Robust fetch title: qui est esse


### 7) JSON shape validation (optional but useful, 10–12 min)

In [14]:
# Quickly assert expected keys so your agent doesn’t break downstream.

def validate_post(obj:dict):
    required = {"userId","id","title","body"}
    missing = required - obj.keys()
    assert not missing, f"Missing keys: {missing}"
 
validate_post(data)  # raises if shape changes

### 8) Mini-project: API wrapper your agent can call (10–15 min)

In [16]:
# Expose a tiny, reusable function your future agent can use.

def search_github_issues(repo_fullname:str, q:str, per_page:int=5):
    url = "https://api.github.com/search/issues"
    params = {"q": f"repo:{repo_fullname} {q}", "per_page": per_page, "sort":"created"}
    return robust_get(url, headers=headers, params=params)
 
results = search_github_issues("pallets/flask", "bug")
for item in results.get("items", []):
    print("-", item["title"])
    
# Extend later: add caching, rate-limit awareness, and structured summaries for agent consumption.

- Test failures with click 8.3.1
- Document 415 on the receiving json section
- prosper_web.py
- pass context through dispatch methods
- merge app and request context
